In [1]:
import bs4
import requests
import pandas as pd
from datetime import datetime
import time
import os

species = {70: "boomklever", 77: "bosuil", 116: "halsbandparkiet", 300: "oehoe", 362: "zwarte ooievaar", 10041: "gewone wasbeer"}

# Base URL for scraping
base_url = 'https://waarnemingen.be/species/{}/observations/?date_after=1900-01-01&date_before=2024-12-31&page={}'  # te verwachten tot page 3879

# Function to parse a single page
def parse_page(species_id, page_number, retries=10, backoff_factor=2):
    for attempt in range(retries):
        try:
            res = requests.get(base_url.format(species_id, page_number))
            res.raise_for_status()
            soup = bs4.BeautifulSoup(res.text, 'html.parser')
            
            # Extract table rows
            rows = soup.select('table tbody tr')
            observations = []
            
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 5:
                    observation = {
                        'date': cols[0].getText().strip(),
                        'id': cols[0].a['href'].strip('/observation/').strip('/') if cols[0].a else None,
                        'amount_manner': cols[1].getText().strip(),
                        'location': cols[2].getText().strip(),
                        'location_id': cols[2].a['href'].strip('/locations/').strip('/') if cols[2].a else None,
                        'observer_name': cols[3].getText().strip(),
                        'observer_id': cols[3].a['href'].strip('/users/').strip('/') if cols[3].a else None,
                        'validation': cols[4].i['title'].strip() if cols[4].i else None,
                        'page' : page_number
                    }
                    observations.append(observation)
                    
            pagination = soup.select('.pagination li')

            has_next_page = False
            for page in pagination:
                text = page.getText()
                try:
                    number = int(text)
                    if number > page_number:
                        has_next_page = True
                        break
                except ValueError:
                    continue
            return observations, has_next_page
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error: {e} on attempt {attempt + 1}/{retries}")
            if attempt < retries - 1:
                time.sleep(backoff_factor * (2 ** attempt))
                continue
            else:
                raise e

def scrape(species_id, page_start = 1, page_end_incl = 100_000):
    file_name = make_filename(species_id)
    page = page_start
    keep_scraping = True
    first_write = True if page_start == 1 else False  # Add header only for the first page
    
    while keep_scraping:
        print(f"Scraping page {page}...")
        try:
            observations, has_next_page = parse_page(species_id, page)
            if not observations or not has_next_page or page >= page_end_incl:
                keep_scraping = False
            
            # Convert observations to a DataFrame
            if observations:
                df = pd.DataFrame(observations)
                
                # Append observations to CSV
                df.to_csv(
                    file_name,
                    mode='a',  # Append mode
                    index=False,
                    header=first_write  # Write header only for the first write
                )
                first_write = False  # Ensure header is only written once
            
            time.sleep(5)  # Respectful delay between requests
            page += 1
        except Exception as e:
            print(f"An error occurred: {e}")
            break
    df = pd.read_csv(file_name)
    print(f"Scraped {len(df)} observations, last scraped page: {page - 1}")
    os.rename(file_name, make_filename(species_id, page_start, page - 1))
    return df

def make_filename(species_id, page_start = None, page_end_incl = None):
    current_date = datetime.now().strftime('%Y-%m-%d')
    base_name = f'scraped_data/observations_{species_id}_{current_date}'
    if page_start and page_end_incl:
        base_name = base_name + f'_pages_{page_start}-{page_end_incl}'
    return base_name + '.csv'

#################################################################################################################

species_id = 70
page_start = 10919
page_end_incl = 2

# scrape(species_id) # Scrape all pages
scrape(species_id, page_start) # Scrape all pages beginning from 
# scrape(species_id, page_start, page_end_incl) # Scrape section

print("Scraping complete. Data saved to .csv file")


Scraping page 10919...
Scraping page 10920...
Scraping page 10921...
Scraping page 10922...
Scraping page 10923...
Scraping page 10924...
Scraping page 10925...
Scraping page 10926...
Scraping page 10927...
Scraping page 10928...
Scraping page 10929...
Scraping page 10930...
Scraping page 10931...
Scraping page 10932...
Scraping page 10933...
Scraping page 10934...
Scraping page 10935...
Scraping page 10936...
Scraping page 10937...
Scraping page 10938...
Scraping page 10939...
Scraping page 10940...
Scraping page 10941...
Scraping page 10942...
Scraping page 10943...
Scraping page 10944...
Scraping page 10945...
Scraping page 10946...
Scraping page 10947...
Scraping page 10948...
Scraping page 10949...
Scraping page 10950...
Scraping page 10951...
Scraping page 10952...
Scraping page 10953...
Scraping page 10954...
Scraping page 10955...
Scraping page 10956...
Scraping page 10957...
Scraping page 10958...
Scraping page 10959...
Scraping page 10960...
Scraping page 10961...
Scraping pa